# Linear Programming

In [1]:
import numpy as np
import pandas as pd
import random
import warnings
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression
import plotly.express as px

In [3]:
# import data
#centroid_locations = pd.read_csv(r"C:\Users\dodie\Downloads\spring 2024\trio-capstone\data\CensusTractCentroids.csv")
#metro_locations = pd.read_csv(r"C:\Users\dodie\Downloads\spring 2024\trio-capstone\data\MetroLinkStations_REGISTERED.csv")
#north_south_locations = pd.read_csv(r"C:\Users\dodie\Downloads\spring 2024\trio-capstone\data\MetroLinkStations_NS.csv")
#census_data = data = pd.read_csv(r"C:\Users\dodie\Downloads\spring 2024\trio-capstone\data\B08119_stl_city.csv")
#race_data = pd.read_csv(r"C:\Users\dodie\Downloads\spring 2024\trio-capstone\data\B08105_stl_city.csv")

centroid_locations = pd.read_csv("/Users/dylanmack/Library/CloudStorage/OneDrive-WashingtonUniversityinSt.Louis/ESE 499/trio-capstone/data/CensusTractCentroids.csv")
metro_locations = pd.read_csv("/Users/dylanmack/Library/CloudStorage/OneDrive-WashingtonUniversityinSt.Louis/ESE 499/trio-capstone/data/MetroLinkStations_REGISTERED.csv")
north_south_locations = pd.read_csv("/Users/dylanmack/Library/CloudStorage/OneDrive-WashingtonUniversityinSt.Louis/ESE 499/trio-capstone/data/MetroLinkStations_NS.csv")
census_data = data = pd.read_csv("/Users/dylanmack/Library/CloudStorage/OneDrive-WashingtonUniversityinSt.Louis/ESE 499/trio-capstone/data/B08119_stl_city.csv")
race_data = pd.read_csv("/Users/dylanmack/Library/CloudStorage/OneDrive-WashingtonUniversityinSt.Louis/ESE 499/trio-capstone/data/B08105_stl_city.csv")

In [4]:
# clean census data

warnings. filterwarnings('ignore')

# Only keep important columns
census_data = census_data[['location','label','estimate','moe']]
census_data['label'] = census_data['label'].str.replace('Estimate', '').str.replace('Total', '').str.replace('!!', '').str.replace(':', '')
census_data['location'] = census_data['location'].str.replace('; St. Louis city; Missouri','')

# Filter out aggregate columns
all_totals = census_data[census_data['label']=='']
trans_totals_mask = ~census_data['label'].astype(str).str.contains('[$]')
trans_totals = census_data.loc[trans_totals_mask][1:]
income_totals = census_data[census_data['label'].str.startswith("$")]
not_to_include = pd.concat([trans_totals, income_totals, all_totals])
merged_df = pd.merge(census_data, not_to_include, how='left', indicator=True)
no_totals = merged_df[merged_df['_merge'] == 'left_only'].drop(columns=['_merge'])[1:]
split_data = no_totals['label'].str.split('$', 1, expand=True)
no_totals[['transportation', 'income']] = split_data
no_totals.drop(columns=['label'], inplace=True)
no_totals['income'] = '$' + no_totals['income']
from_this = no_totals['income'].unique()
to_this = [1, 10000, 15000, 25000, 35000, 50000, 65000, 75000]
no_totals['avg income'] = no_totals['income'].replace(from_this, to_this)
clean_census_data = no_totals

In [5]:
# clean race data

warnings. filterwarnings('ignore')

# Only keep important columns
race_data = race_data[['location','label','concept','estimate','moe']]
race_data['label'] = race_data['label'].str.replace('Estimate', '').str.replace('Total', '').str.replace('!!', '').str.replace(':', '')
race_data['location'] = race_data['location'].str.replace('; St. Louis city; Missouri','')
race_data['concept'] = race_data['concept'].str.replace('MEANS OF TRANSPORTATION TO WORK ', '').str.replace(')','').str.replace('(','').str.replace('ALONE','')

# Filter out aggregate rows and rename columns
race_data = race_data[race_data['label'] != '']
race_data = race_data.rename(columns={'label': 'transportation', 'concept': 'race'})

In [6]:
# Group by census tract and mode of transportation
by_tract_trans = clean_census_data.groupby(['location','transportation'])['estimate'].sum()

# Find percentage of people who take public transport in each tract
tract_pops = clean_census_data.groupby('location', as_index = False)['estimate'].sum()['estimate'].tolist()
public_transit_pops = clean_census_data[clean_census_data['transportation']=='Public transportation (excluding taxicab)'].groupby(['location','transportation'], as_index = False)['estimate'].sum()['estimate'].tolist()

pct_public_trans = []
for i in range(len(tract_pops)):
    pct_public_trans.append(public_transit_pops[i]/tract_pops[i])

# Add percentages back in to groupby
by_tract = clean_census_data.groupby('location', as_index = False)['estimate'].sum()
by_tract['public transport count'] = public_transit_pops
by_tract['public transport %'] = pct_public_trans

# Group by census tract and income
by_tract_inc = clean_census_data.groupby(['location','avg income'])['estimate'].sum()

# Find average income of each tract
tract_incomes = (clean_census_data['avg income']*clean_census_data['estimate']).groupby(clean_census_data['location']).sum().tolist()
tract_totals = sum(tract_incomes)
tract_incomes_new = np.array([tract_totals - i for i in tract_incomes])
income_weight = tract_incomes_new / sum(tract_incomes_new)

# Add percentages back in to groupby
by_tract['income weight'] = income_weight

# Find minority (nonwhite) population of each tract
total_pops = race_data.groupby('location')['estimate'].sum().tolist()
nonwhite_pops = race_data[race_data['race'] != 'WHITE '].groupby('location')['estimate'].sum().tolist()

# add back in to groupby
by_tract['race weight'] = np.divide(np.array(nonwhite_pops), np.array(total_pops))

In [7]:
# merge census tract location data with census data

centroid_locations_small = centroid_locations[['NAMELSAD','INTPTLAT','INTPTLON']]
centroid_locations_small = centroid_locations_small.rename(columns = {"NAMELSAD": "location", "INTPTLAT" : "LAT", "INTPTLON" : "LON"})

full_data = by_tract.merge(centroid_locations_small, on='location')
full_data.head()

,location,estimate,public transport count,public transport %,income weight,race weight,LAT,LON
0,Census Tract 1011,1039,31,0.029836,0.009653,0.538604,38.554718,-90.273620
1,Census Tract 1012,1921,11,0.005726,0.009570,0.570798,38.561824,-90.274863
2,Census Tract 1013,2161,35,0.016196,0.009560,0.587548,38.566560,-90.264211
3,Census Tract 1014,1502,95,0.063249,0.009614,0.679515,38.562243,-90.253522
4,Census Tract 1015,1318,32,0.024279,0.009645,0.582895,38.548544,-90.264511


In [8]:
north_south_locations.head()

,id,Name,LON,LAT
0,1,Grand Blvd/Fairground Park,-90.217,38.661
1,2,Palm St/Salisbury St,-90.209,38.658
2,3,St Louis Ave,-90.211,38.653
3,4,Cass Ave,-90.213,38.645
4,5,Carr St,-90.213,38.640


In [9]:
k = 12
n = len(full_data)
x0 = []
for i in range(k):
    new_tract = random.randint(0,n-1)
    x0.append(full_data['LON'][new_tract])
    x0.append(full_data['LAT'][new_tract])

In [10]:
lat_bounds = (min(full_data['LAT']),max(full_data['LAT']))
lon_bounds = (min(full_data['LON']),max(full_data['LON']))
bnds = [val for pair in zip([lon_bounds]*12, [lat_bounds]*12) for val in pair]

In [11]:
def distances_to_nearest_stop(tract_locations, metro_locations):
    
    shortest_distance = np.zeros(len(tract_locations))
    metro_locations = [[metro_locations[i], metro_locations[i + 1]] for i in range(0, len(metro_locations), 2)] # reshape back into 2d array
    
    for i in range(len(tract_locations)):
        cent = np.array([tract_locations['LON'][i],tract_locations['LAT'][i]])
        closest_distance = float('inf') # initialize to infinity
        for j in range(len(metro_locations)):
            metro = np.array(metro_locations[j])
            distance = np.linalg.norm(cent-metro)
            if (distance < closest_distance):
                closest_distance = distance
        shortest_distance[i] = closest_distance
    
    return shortest_distance

In [12]:
def orth_dist(x):
    x = [[x[i], x[i + 1]] for i in range(0, len(x), 2)] # reshape back into 2d array
    model = LinearRegression().fit(np.array(x[0][:]).reshape(-1,1),np.array(x[1][:]))
    b = model.intercept_
    m = model.coef_[0]
    dists = 0
    for stop in x:
        n = abs(-1*m*stop[0]+stop[1]-b)
        d = np.sqrt(m**2+1)
        dists = dists + n/d
    return dists

In [13]:
def distances_between_stops(metro_locations):
    metro_locations = [[metro_locations[i], metro_locations[i + 1]] for i in range(0, len(metro_locations), 2)] # reshape back into 2d array
    shortest_distance = np.zeros(len(metro_locations))
    total_distance = 0
    for i in range(len(metro_locations)):
        metro_current = np.array([metro_locations[i][0],metro_locations[i][1]])
        closest_distance = float('inf') # initialize to infinity
        for j in range(len(metro_locations)):
            if (i != j):
                metro_compared = np.array(metro_locations[j])
                distance = np.linalg.norm(metro_current-metro_compared)
                if (distance < closest_distance):
                    closest_distance = distance
        total_distance = total_distance + closest_distance
    return total_distance

In [14]:
def distance_to_mean_x(metro_locations):
    metro_locations = [[metro_locations[i], metro_locations[i + 1]] for i in range(0, len(metro_locations), 2)] # reshape back into 2d array
    x_avg = np.mean(metro_locations[:][0])
    deviation_from_avg = 0
    for i in range(len(metro_locations)):
        deviation_from_avg = deviation_from_avg + np.abs(x_avg - metro_locations[i][0])
    return deviation_from_avg

In [27]:
def fun(new_stops):
    #print(f'shape of new stops: {np.shape(new_stops)}')
    #weights = full_data['race weight']
    weights = np.ones(len(full_data))
    #print(weights)
    dist_to_stops = np.dot(weights, distances_to_nearest_stop(full_data, new_stops))
    linearity = orth_dist(new_stops) 
    dist_btw_stops = distances_between_stops(new_stops)
    distance_to_mean = distance_to_mean_x(new_stops)
    print(f'dist to stops: {dist_to_stops:3.3f}, linearity: {linearity:3.3f}')
    return 5*dist_to_stops + 1.0*linearity - 1.0*dist_btw_stops + 1*distance_to_mean

In [28]:
result = minimize(fun, x0, bounds=bnds)
all_centroids = result.x
lp_results = [[all_centroids[i], all_centroids[i + 1]] for i in range(0, len(all_centroids), 2)]


dist to stops: 2.014, linearity: 1093.338
dist to stops: 2.014, linearity: 1093.338
dist to stops: 2.014, linearity: 1093.338
dist to stops: 2.014, linearity: 1093.338
dist to stops: 2.014, linearity: 1093.338
dist to stops: 2.014, linearity: 1093.338
dist to stops: 2.014, linearity: 1093.338
dist to stops: 2.014, linearity: 1093.338
dist to stops: 2.014, linearity: 1093.338
dist to stops: 2.014, linearity: 1093.338
dist to stops: 2.014, linearity: 1093.338
dist to stops: 2.014, linearity: 1093.338
dist to stops: 2.014, linearity: 1093.338
dist to stops: 2.014, linearity: 1093.338
dist to stops: 2.014, linearity: 1093.338
dist to stops: 2.014, linearity: 1093.338
dist to stops: 2.014, linearity: 1093.338
dist to stops: 2.014, linearity: 1093.338
dist to stops: 2.014, linearity: 1093.338
dist to stops: 2.014, linearity: 1093.338
dist to stops: 2.014, linearity: 1093.338
dist to stops: 2.014, linearity: 1093.338
dist to stops: 2.014, linearity: 1093.338
dist to stops: 2.014, linearity: 1

dist to stops: 1.440, linearity: 1093.569
dist to stops: 1.440, linearity: 1093.569
dist to stops: 1.440, linearity: 1093.569
dist to stops: 1.440, linearity: 1093.569
dist to stops: 1.440, linearity: 1093.569
dist to stops: 1.440, linearity: 1093.569
dist to stops: 1.440, linearity: 1093.569
dist to stops: 1.440, linearity: 1093.569
dist to stops: 1.440, linearity: 1093.569
dist to stops: 1.440, linearity: 1093.569
dist to stops: 1.440, linearity: 1093.569
dist to stops: 1.440, linearity: 1093.569
dist to stops: 1.440, linearity: 1093.569
dist to stops: 1.440, linearity: 1093.569
dist to stops: 1.440, linearity: 1093.569
dist to stops: 1.440, linearity: 1093.569
dist to stops: 1.415, linearity: 1093.585
dist to stops: 1.415, linearity: 1093.585
dist to stops: 1.415, linearity: 1093.585
dist to stops: 1.415, linearity: 1093.585
dist to stops: 1.415, linearity: 1093.585
dist to stops: 1.415, linearity: 1093.585
dist to stops: 1.415, linearity: 1093.585
dist to stops: 1.415, linearity: 1

dist to stops: 1.382, linearity: 1093.597
dist to stops: 1.382, linearity: 1093.597
dist to stops: 1.382, linearity: 1093.597
dist to stops: 1.382, linearity: 1093.597
dist to stops: 1.382, linearity: 1093.597
dist to stops: 1.382, linearity: 1093.597
dist to stops: 1.382, linearity: 1093.597
dist to stops: 1.382, linearity: 1093.597
dist to stops: 1.383, linearity: 1093.596
dist to stops: 1.383, linearity: 1093.596
dist to stops: 1.383, linearity: 1093.596
dist to stops: 1.383, linearity: 1093.596
dist to stops: 1.383, linearity: 1093.596
dist to stops: 1.383, linearity: 1093.596
dist to stops: 1.383, linearity: 1093.596
dist to stops: 1.383, linearity: 1093.596
dist to stops: 1.383, linearity: 1093.596
dist to stops: 1.383, linearity: 1093.596
dist to stops: 1.383, linearity: 1093.596
dist to stops: 1.383, linearity: 1093.596
dist to stops: 1.383, linearity: 1093.596
dist to stops: 1.383, linearity: 1093.596
dist to stops: 1.383, linearity: 1093.596
dist to stops: 1.383, linearity: 1

dist to stops: 1.382, linearity: 1093.598
dist to stops: 1.382, linearity: 1093.598
dist to stops: 1.382, linearity: 1093.598
dist to stops: 1.382, linearity: 1093.598
dist to stops: 1.382, linearity: 1093.598
dist to stops: 1.382, linearity: 1093.598
dist to stops: 1.382, linearity: 1093.598
dist to stops: 1.382, linearity: 1093.598
dist to stops: 1.382, linearity: 1093.598
dist to stops: 1.382, linearity: 1093.598
dist to stops: 1.382, linearity: 1093.598
dist to stops: 1.382, linearity: 1093.598
dist to stops: 1.382, linearity: 1093.598
dist to stops: 1.382, linearity: 1093.598
dist to stops: 1.382, linearity: 1093.598
dist to stops: 1.382, linearity: 1093.598
dist to stops: 1.382, linearity: 1093.598
dist to stops: 1.382, linearity: 1093.598
dist to stops: 1.384, linearity: 1093.598
dist to stops: 1.384, linearity: 1093.598
dist to stops: 1.384, linearity: 1093.598
dist to stops: 1.384, linearity: 1093.598
dist to stops: 1.384, linearity: 1093.598
dist to stops: 1.384, linearity: 1

dist to stops: 1.382, linearity: 1093.598
dist to stops: 1.382, linearity: 1093.597
dist to stops: 1.382, linearity: 1093.597
dist to stops: 1.382, linearity: 1093.597
dist to stops: 1.382, linearity: 1093.597
dist to stops: 1.382, linearity: 1093.597
dist to stops: 1.382, linearity: 1093.597
dist to stops: 1.382, linearity: 1093.597
dist to stops: 1.382, linearity: 1093.597
dist to stops: 1.382, linearity: 1093.597
dist to stops: 1.382, linearity: 1093.597
dist to stops: 1.382, linearity: 1093.597
dist to stops: 1.382, linearity: 1093.597
dist to stops: 1.382, linearity: 1093.597
dist to stops: 1.382, linearity: 1093.597
dist to stops: 1.382, linearity: 1093.597
dist to stops: 1.382, linearity: 1093.597
dist to stops: 1.382, linearity: 1093.597
dist to stops: 1.382, linearity: 1093.597
dist to stops: 1.382, linearity: 1093.597
dist to stops: 1.382, linearity: 1093.597
dist to stops: 1.382, linearity: 1093.597
dist to stops: 1.382, linearity: 1093.597
dist to stops: 1.382, linearity: 1

In [29]:
print(f'Final Objective Function Value: {result.fun}')

def dist_to_nearest_stop_eval(centroid_locations, metro_locations):
    shortest_distance = np.zeros(len(centroid_locations))
    
    for i in range(len(centroid_locations)):
        cent = np.array([centroid_locations.INTPTLAT[i],centroid_locations.INTPTLON[i]])
        closest_distance = float('inf') # initialize to infinity
        for j in range(len(metro_locations)):
            metro = np.array([metro_locations[j][1],metro_locations[j][0]])
            distance = np.linalg.norm(cent-metro)
            if (distance < closest_distance):
                closest_distance = distance
        shortest_distance[i] = closest_distance
    
    return shortest_distance

shortest_distance = dist_to_nearest_stop_eval(centroid_locations, lp_results)
print(f'Sum of Nearest Stop Evaluation: {np.sum(shortest_distance)}')


Final Objective Function Value: 1873.811296717634
Sum of Nearest Stop Evaluation: 1.3823044676633662


In [30]:
lp_results = [[all_centroids[i], all_centroids[i + 1]] for i in range(0, len(all_centroids), 2)]

cluster_centroids = pd.DataFrame(data = lp_results, columns = ["long", "lat"])
cluster_centroids['size'] = [1 for i in range(len(cluster_centroids))]

fig = px.scatter_mapbox(cluster_centroids,
                        lat='lat',
                        lon='long',
                        size='size',
                        zoom=10,
                        height=750)
fig.update_layout(mapbox_style="open-street-map")

In [ ]:
## PARAMETER SWEEPING

a = np.linspace(0, 100, 5)
b = np.linspace(0, 25, 5)
c = np.linspace(0, 25, 5)
d = np.linspace(0, 25, 5)

obj_fun_values = np.zeros([len(a)*len(b)*len(c)*len(d), 5])

def fun_sweep(new_stops):
    #print(f'shape of new stops: {np.shape(new_stops)}')
    #weights = full_data['race weight']
    weights = np.ones(len(full_data))
    #print(weights)
    dist_to_stops = np.dot(weights, distances_to_nearest_stop(full_data, new_stops))
    linearity = orth_dist(new_stops) 
    dist_btw_stops = distances_between_stops(new_stops)
    distance_to_mean = distance_to_mean_x(new_stops)
    #print(f'dist to stops: {dist_to_stops:3.3f}, linearity: {linearity:3.3f}')
    return i*dist_to_stops + j*linearity - k*dist_btw_stops + l*distance_to_mean

counter = 0
for i in a:
    for j in b:
        for k in c:
            for l in d:
                result = minimize(fun_sweep, x0, bounds=bnds)
                obj_fun_values[counter, 0] = i
                obj_fun_values[counter, 1] = j
                obj_fun_values[counter, 2] = k
                obj_fun_values[counter, 3] = l
                obj_fun_values[counter, 4] = result.fun
                counter = counter + 1
                

In [ ]:
counter



In [ ]:
def fun_opt_coefs(new_stops, a, b, c, d):
    #print(f'shape of new stops: {np.shape(new_stops)}')
    #weights = full_data['race weight']
    weights = np.ones(len(full_data))
    #print(weights)
    dist_to_stops = np.dot(weights, distances_to_nearest_stop(full_data, new_stops))
    linearity = orth_dist(new_stops) 
    dist_btw_stops = distances_between_stops(new_stops)
    distance_to_mean = distance_to_mean_x(new_stops)
    #print(f'dist to stops: {dist_to_stops:3.3f}, linearity: {linearity:3.3f}')
    return a*dist_to_stops + b*linearity - c*dist_btw_stops + d*distance_to_mean

result = minimize(fun_opt_coefs, x0, bounds=bnds)
all_centroids = result.x